In [3]:
import pandas as pd
from pathlib import Path

## Fetch Data

In [4]:
import utils.data_utils as data_utils

DATA_DIR = Path.cwd().parent / "data"
train, test = data_utils.load_swift_data(DATA_DIR)
bank_data = data_utils.load_bank_data(DATA_DIR)

In [7]:
train.head()

,Timestamp,UETR,Sender,Receiver,TransactionReference,OrderingAccount,OrderingName,OrderingStreet,OrderingCountryCityZip,BeneficiaryAccount,BeneficiaryName,BeneficiaryStreet,BeneficiaryCountryCityZip,SettlementDate,SettlementCurrency,SettlementAmount,InstructedCurrency,InstructedAmount,Label
MessageId,,,,,,,,,,,,,,,,,,,
TRA7CGN3FF,2022-01-01,f474fdb3-4675-4fff-ab7e-3469f82bd6a7,DPSUFRPP,ABVVUS6S,PETX22-FXIDA-7054,FR90714755422956984353,PHACELIA HETEROPHYLLA,3| RUE HAMON,FR/42859 SAINTE AURÉLIE,611024064274704358,PAPAVER CALIFORNICUM,2584 CHARLES PLACE,US/ROJASLAND| DC 58442,220101,USD,1.746319e+09,EUR,1.560189e+09,0
TRPNEMZIR7,2022-01-01,c9158def-dab1-4bfb-a31f-7f51c6679d60,BRRGPTPL,CBLHESMM,PETX22-NO-FX-1736,PT8895792452733129969,GONOLOBUS STEPHANOTRICHUS,AV RITA ALVES| 60,PT/5863-752 CANTANHEDE,ES61897100852916932423,MINUARTIA NUTTALLII-GREGARIA,ACCESO DE CARMINA ARAGÓN 83 PUERTA 4,ES/ÁVILA| 02281,220101,EUR,4.711420e+06,EUR,4.711420e+06,0
TR6S6A5JYL,2022-01-01,d371ba0a-823f-4243-98ba-94ff18523420,BRRGPTPL,CBLHESMM,PETX22-NO-FX-1687,PT92895792452733126420,LECHEA INTERMEDIA-INTERMEDIA,PRAÇA VALENTE| 85,PT/1100-087 BARCELOS,ES31897100852916935097,ASTRAGALUS MAGDALENAE,PASADIZO ANÍBAL LUJÁN 57,ES/SEGOVIA| 40727,220101,EUR,7.528216e+05,EUR,7.528216e+05,0
TRT4VPEHAK,2022-01-01,5a53a257-4dc9-4800-abb2-4cd1d55c8345,DPSUFRPP,ABVVUS6S,DPSU22-FXIYA-517,358727697099645998,SCLERANTHUS,341 4 CHOME 4 BAN 2 GO,JP/FUKUOKA PREFECTURE|ŌKAWA,611024064274698543,SYNGONIUM PODOPHYLLUM,7864 MORRIS MEWS APT. 464,US/DPO AE 78549,220101,USD,6.371209e+06,JPY,6.490487e+08,0
TRTADQPGAV,2022-01-01,f27867ac-35e2-46af-8248-0a2d0d9bf00d,DPSUFRPP,ABVVUS6S,PETX22-FXIDA-11878,FR71714755422956985471,SELAGINELLA ASPRELLA,28| BOULEVARD LÉVÊQUE,FR/36357 TURPIN,611024064274707099,GALACTIA PARVIFOLIA,363 ROBERT GARDENS,US/NEW KAREN| MS 49461,220101,USD,5.179423e+06,EUR,4.627377e+06,0


In [10]:
bank_data.head()

,Bank,Account,Name,Street,CountryCityZip,Flags
0,ABVVUS6S,611024064274690946,OXYTROPIS PODOCARPA,784 YANG PASSAGE,"US/NEW SHANNON, IA 49432",0
1,ABVVUS6S,611024064274690947,VIOLA ◊ESCULENTA,786 CINDY HARBOR,US/APO AP 02202,0
2,ABVVUS6S,611024064274690948,CROSSOSOMA CALIFORNICUM,235 JAMES VIEWS SUITE 206,"US/ORRVILLE, LA 49084",0
3,ABVVUS6S,611024064274690949,HYDROCOTYLE PROLIFERA,270 THERESA MILL,"US/WEST JAYSHIRE, WA 67409",0
4,ABVVUS6S,611024064274690950,CHAMAEMELUM FUSCATUM,20734 ISAIAH BURG,"US/KINGVIEW, CO 71032",0


### merge data

In [9]:
bank_data[bank_data['Account'] == 'PT92895792452733126420']

,Bank,Account,Name,Street,CountryCityZip,Flags
29115,BRRGPTPL,PT92895792452733126420,LECHEA INTERMEDIA-INTERMEDIA,"PRAÇA VALENTE, 85",PT/1100-087 BARCELOS,0
